In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 
import operator

In [2]:
N=20
nK=3
df={}
for i in range(N):
    df[i,'x']=random.random()
    df[i,'y']=random.random()


In [3]:
n=int(len(df)/2)
model = AbstractModel()
model.N =Param(mutable=True, default=n) 
model.i = RangeSet(1,model.N)
model.k = RangeSet(1,nK)
model.j = Set(initialize=model.i)
model.Alpha = Var(model.k,bounds=(-2,2), within=Reals)
model.Beta = Var(model.k, bounds=(-2,2), within=Reals)
model.U = Var(model.i,model.k, initialize=0,bounds=(0,1), within=Binary)
model.r = Var(model.i,bounds=(0,200), within=NonNegativeReals)


def rule_C1(model,i,k):
        return model.r[i]>= df[i-1,'x']*model.Alpha[k]+model.Beta[k] -df[i-1,'y'] - 4*(1-model.U[i,k])
model.C1   = Constraint(model.i,model.k,rule=rule_C1)

def rule_C2(model,i,k):
        return model.r[i]>= -(df[i-1,'x']*model.Alpha[k]+model.Beta[k] -df[i-1,'y']) - 4*(1-model.U[i,k])
model.C2   = Constraint(model.i,model.k,rule=rule_C2)

def rule_C7(model,i):
        return sum(model.U[i,k] for k in model.k)==1
model.C7   = Constraint(model.i,rule=rule_C7)

def rule_C9(model,k):
        if k+1<=nK:
            return model.Alpha[k]<= model.Alpha[k+1]
        else:
            return Constraint.Skip
model.C9   = Constraint(model.k,rule=rule_C9)

def rule_OF(model):
    return sum(model.r[i] for i in model.i)
model.obj1 = Objective(rule=rule_OF, sense=minimize)

In [ ]:
opt = SolverFactory('glpk')


model.N=n
instance = model.create_instance()

results = opt.solve(instance) # solves and updates instance
print('OF= ',value(instance.obj1))
print ("The solver returned a status of:"+str(results.solver.status))
from pyomo.opt import SolverStatus, TerminationCondition
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
     print ("this is feasible and optimal")
elif results.solver.termination_condition == TerminationCondition.infeasible:
     print ("do something about it? or exit?")
else:
     print (str(results.solver))